In [2]:
import numpy as np
import matplotlib.pyplot as plt
import gc
#from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas

def msj(k,width):
    if (k+1)%width > 0:
        print(k, end='Ϟ ')
    else:
        print(k,'Ϟ',end='\r')

fig = plt.figure(figsize=(4,4),dpi=250)
ax = fig.add_subplot(111)

for k in range(4000):
    x = np.random.rand(100,)
    y = np.random.rand(100,)
    ax.scatter(x,y)
    filename = 'prueba'+str(k)+'.png'
    fig.savefig(filename)
    ax.cla()
    fig.clf()
    msj(k,20) # Esto no importa, nada más es para imprimir dónde voy

<Figure size 1000x1000 with 0 Axes>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [6]:
!pip install mido

     |████████████████████████████████| 61kB 1.9MB/s 


In [7]:
import numpy as np
import mido
import os
import pickle
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.use('Agg')
import pandas as pd

# Clase

In [ ]:
from matplotlib.colors import ColorConverter
# inherit the origin mido class
class MidiFile(mido.MidiFile):

    def __init__(self, filename):
        mido.MidiFile.__init__(self, filename)
        self.sr = 10
        self.meta = {}
        self.events = self.get_events()

    def get_events(self):
        mid = self
        # There is > 16 channel in midi.tracks. However there is only 16 channel related to "music" events.
        # We store music events of 16 channel in the list "events" with form [[ch1],[ch2]....[ch16]]
        # Lyrics and meta data used a extra channel which is not include in "events"
        events = [[] for x in range(16)]
        # Iterate all event in the midi and extract to 16 channel form
        for track in mid.tracks:
            for msg in track:
                try:
                    channel = msg.channel
                    events[channel].append(msg)
                except AttributeError:
                    try:
                        if type(msg) != type(mido.UnknownMetaMessage):
                            self.meta[msg.type] = msg.dict()
                        else:
                            pass
                    except:
                        print("error",type(msg))
        return events

    def get_roll(self):
        events = self.get_events()
        # Identify events, then translate to piano roll
        # choose a sample ratio(sr) to down-sample through time axis
        sr = self.sr
        # compute total length in tick unit
        length = self.get_total_ticks()
        # allocate memory to numpy array
        roll = np.zeros((16, 128, length // sr), dtype="int8")
        # use a register array to save the state(no/off) for each key
        note_register = [int(-1) for x in range(128)]
        # use a register array to save the state(program_change) for each channel
        timbre_register = [1 for x in range(16)]

        for idx, channel in enumerate(events):
            time_counter = 0
            volume = 100
            # Volume would change by control change event (cc) cc7 & cc11
            # Volume 0-100 is mapped to 0-127
            # print("channel", idx, "start")
            for msg in channel:
                if msg.type == "control_change":
                    if msg.control == 7:
                        volume = msg.value
                        # directly assign volume
                    if msg.control == 11:
                        volume = volume * msg.value // 127
                        # change volume by percentage
                    # print("cc", msg.control, msg.value, "duration", msg.time)
                if msg.type == "program_change":
                    timbre_register[idx] = msg.program
                    #print("channel", idx, "pc", msg.program, "time", time_counter, "duration", msg.time)

                if msg.type == "note_on":
                    #print("on ", msg.note, "time", time_counter, "duration", msg.time, "velocity", msg.velocity)
                    note_on_start_time = time_counter // sr
                    note_on_end_time = (time_counter + msg.time) // sr
                    intensity = volume * msg.velocity // 127
					# When a note_on event *ends* the note start to be play 
					# Record end time of note_on event if there is no value in register
					# When note_off event happens, we fill in the color
                    if note_register[msg.note] == -1:
                        note_register[msg.note] = (note_on_end_time,intensity)
                    else:
					# When note_on event happens again, we also fill in the color
                        old_end_time = note_register[msg.note][0]
                        old_intensity = note_register[msg.note][1]
                        roll[idx, msg.note, old_end_time: note_on_end_time] = old_intensity
                        note_register[msg.note] = (note_on_end_time,intensity)

                if msg.type == "note_off":
                    #print("off", msg.note, "time", time_counter, "duration", msg.time, "velocity", msg.velocity)
                    note_off_start_time = time_counter // sr
                    note_off_end_time = (time_counter + msg.time) // sr
                    note_on_end_time = note_register[msg.note][0]
                    intensity = note_register[msg.note][1]
					# fill in color
                    roll[idx, msg.note, note_on_end_time:note_off_end_time] = intensity
                    note_register[msg.note] = -1  # reinitialize register
                time_counter += msg.time

            # if there is a note not closed at the end of a channel, close it
            for key, data in enumerate(note_register):
                if data != -1:
                    note_on_end_time = data[0]
                    intensity = data[1]
                    # print(key, note_on_end_time)
                    note_off_start_time = time_counter // sr
                    roll[idx, key, note_on_end_time:] = intensity
                note_register[idx] = -1
        return roll

    def get_roll_image(self):
        roll = self.get_roll()
        plt.ioff()
        K = 16
        transparent = ColorConverter.to_rgba('black')
        colors = [mpl.colors.to_rgba(mpl.colors.hsv_to_rgb((i / K, 1, 1)), alpha=1) for i in range(K)]
        cmaps = [mpl.colors.LinearSegmentedColormap.from_list('my_cmap', [transparent, colors[i]], 128) for i in
                 range(K)]
        for i in range(K):
            cmaps[i]._init()  # create the _lut array, with rgba values
            # create your alpha array and fill the colormap with them.
            # here it is progressive, but you can create whathever you want
            alphas = np.linspace(0, 1, cmaps[i].N + 3)
            cmaps[i]._lut[:, -1] = alphas

        fig = plt.figure(figsize=(4, 3))
        a1 = fig.add_subplot(111)
        a1.axis("equal")
        a1.set_facecolor("black")
        array = []
        for i in range(K):
            try:
                img = a1.imshow(roll[i], interpolation='nearest', cmap=cmaps[i], aspect='auto')
                array.append(img.get_array())
            except IndexError:
                pass
        return array

    def draw_roll(self,filename:str,just_image=True,show=True):
        roll = self.get_roll()
        # build and set fig obj
        plt.ioff()
        fig = plt.figure(figsize=(7, 5),frameon=False)
        a1 = fig.add_subplot(111)
        a1.axis("equal")
        a1.set_facecolor("black")

        # change unit of time axis from tick to second
        tick = self.get_total_ticks()
        second = mido.tick2second(tick, self.ticks_per_beat, self.get_tempo())
        #print(second)
        if second > 10:
            x_label_period_sec = second // 10
        else:
            x_label_period_sec = second / 10  # ms
        #print(x_label_period_sec)
        x_label_interval = mido.second2tick(x_label_period_sec, self.ticks_per_beat, self.get_tempo()) / self.sr
        #print(x_label_interval)
        if just_image==False:                
            plt.xticks([int(x * x_label_interval) for x in range(20)], [round(x * x_label_period_sec, 2) for x in range(20)])
            # change scale and label of y axis
            plt.yticks([y*16 for y in range(8)], [y*16 for y in range(8)])
        else:
            plt.xticks([])
            plt.yticks([])
        # build colors
        channel_nb = 16
        transparent = ColorConverter.to_rgba('black')
        colors = [mpl.colors.to_rgba(mpl.colors.hsv_to_rgb((i / channel_nb, 1, 1)), alpha=1) for i in range(channel_nb)]
        cmaps = [mpl.colors.LinearSegmentedColormap.from_list('my_cmap', [transparent, colors[i]], 128) for i in
                 range(channel_nb)]
        # build color maps
        for i in range(channel_nb):
            cmaps[i]._init()
            # create your alpha array and fill the colormap with them.
            alphas = np.linspace(0, 1, cmaps[i].N + 3)
            # create the _lut array, with rgba values
            cmaps[i]._lut[:, -1] = alphas

        # draw piano roll and stack image on a1
        for i in range(channel_nb):
            try:
                a1.imshow(roll[i], origin="lower", interpolation='nearest', cmap=cmaps[i], aspect='auto')
                # a1.imshow(roll[i], origin="lower", interpolation='nearest', cmap='inferno', aspect='auto')
            except IndexError:
                pass
        # draw color bar
        # colors = [mpl.colors.hsv_to_rgb((i / channel_nb, 1, 1)) for i in range(channel_nb)]
        # cmap = mpl.colors.LinearSegmentedColormap.from_list('my_cmap', colors, 16)
        # a2 = fig.add_axes([0.05, 0.80, 0.9, 0.15])
        # cbar = mpl.colorbar.ColorbarBase(a2, cmap=cmap,
        #                                 orientation='horizontal',
        #                                 ticks=list(range(16)))
        # show piano roll
        #plt.draw()
        #plt.ion()
        # path = '/content/drive/My Drive/Diplomado Python/Notebooks & Resources/Proyecto Final/Images/'
        # filename = path+'roll_1.png'
        plt.savefig(filename,dpi=200)
        if show:
            plt.show(block=True)

    def get_tempo(self):
        try:
            return self.meta["set_tempo"]["tempo"]
        except:
            return 500000

    def get_total_ticks(self):
        max_ticks = 0
        for channel in range(16):
            ticks = sum(msg.time for msg in self.events[channel])
            if ticks > max_ticks:
                max_ticks = ticks
        return max_ticks

if __name__ == "__main__":
    filename = '/content/drive/My Drive/Diplomado Python/Notebooks & Resources/Proyecto Final/Piano solo midi dataset/Bach/bach_847.mid'
    mid = MidiFile(filename)
    roll = mid.get_roll()
    path = '/content/drive/My Drive/Diplomado Python/Notebooks & Resources/Proyecto Final/Images/'
    filename = path+'roll_1.png'    
    mid.draw_roll(filename)

# Mi clase personalizada

Todas las imagenes de un go

MI CLASE

In [11]:
from matplotlib.colors import ColorConverter

class MidiFile(mido.MidiFile):

    def __init__(self, filename):
        mido.MidiFile.__init__(self, filename)
        self.sr = 10
        self.meta = {}
        self.events = self.get_events()

    def get_events(self):
        mid = self
        events = [[] for x in range(16)]
        # Iterate all event in the midi and extract to 16 channel form
        for track in mid.tracks:
            for msg in track:
                try:
                    channel = msg.channel
                    events[channel].append(msg)
                except AttributeError:
                    try:
                        if type(msg) != type(mido.UnknownMetaMessage):
                            self.meta[msg.type] = msg.dict()
                        else:
                            pass
                    except:
                        print("error",type(msg))
        return events

    def get_roll(self):
        events = self.get_events()
        # Identify events, then translate to piano roll
        # choose a sample ratio(sr) to down-sample through time axis
        sr = self.sr
        # compute total length in tick unit
        length = self.get_total_ticks()
        # allocate memory to numpy array
        roll = np.zeros((16, 128, length // sr), dtype="int8")
        # use a register array to save the state(no/off) for each key
        note_register = [int(-1) for x in range(128)]
        # use a register array to save the state(program_change) for each channel
        timbre_register = [1 for x in range(16)]

        for idx, channel in enumerate(events):
            time_counter = 0
            volume = 100
            # Volume would change by control change event (cc) cc7 & cc11
            # Volume 0-100 is mapped to 0-127
            for msg in channel:
                if msg.type == "control_change":
                    if msg.control == 7:
                        volume = msg.value
                        # directly assign volume
                    if msg.control == 11:
                        volume = volume * msg.value // 127
                        # change volume by percentage                   
                if msg.type == "program_change":
                    timbre_register[idx] = msg.program

                if msg.type == "note_on":
                    note_on_start_time = time_counter // sr
                    note_on_end_time = (time_counter + msg.time) // sr
                    intensity = volume * msg.velocity // 127
					# When a note_on event *ends* the note start to be play 
					# Record end time of note_on event if there is no value in register
					# When note_off event happens, we fill in the color
                    if note_register[msg.note] == -1:
                        note_register[msg.note] = (note_on_end_time,intensity)
                    else:
					# When note_on event happens again, we also fill in the color
                        old_end_time = note_register[msg.note][0]
                        old_intensity = note_register[msg.note][1]
                        roll[idx, msg.note, old_end_time: note_on_end_time] = old_intensity
                        note_register[msg.note] = (note_on_end_time,intensity)

                if msg.type == "note_off":
                    note_off_start_time = time_counter // sr
                    note_off_end_time = (time_counter + msg.time) // sr
                    note_on_end_time = note_register[msg.note][0]
                    intensity = note_register[msg.note][1]
					# fill in color
                    roll[idx, msg.note, note_on_end_time:note_off_end_time] = intensity
                    note_register[msg.note] = -1  # reinitialize register
                time_counter += msg.time

            # if there is a note not closed at the end of a channel, close it
            for key, data in enumerate(note_register):
                if data != -1:
                    note_on_end_time = data[0]
                    intensity = data[1]
                    # print(key, note_on_end_time)
                    note_off_start_time = time_counter // sr
                    roll[idx, key, note_on_end_time:] = intensity
                note_register[idx] = -1
        return roll

    def get_roll_image(self):
        roll = self.get_roll()
        plt.ioff()
        K = 16
        transparent = ColorConverter.to_rgba('black')
        colors = [mpl.colors.to_rgba(mpl.colors.hsv_to_rgb((i / K, 1, 1)), alpha=1) for i in range(K)]
        cmaps = [mpl.colors.LinearSegmentedColormap.from_list('my_cmap', [transparent, colors[i]], 128) for i in
                 range(K)]
        for i in range(K):
            cmaps[i]._init()  # create the _lut array, with rgba values
            # create your alpha array and fill the colormap with them.
            # here it is progressive, but you can create whathever you want
            alphas = np.linspace(0, 1, cmaps[i].N + 3)
            cmaps[i]._lut[:, -1] = alphas

        fig = plt.figure(figsize=(4, 3))
        a1 = fig.add_subplot(111)
        a1.axis("equal")
        a1.set_facecolor("black")
        array = []
        for i in range(K):
            try:
                img = a1.imshow(roll[i], interpolation='nearest', cmap=cmaps[i], aspect='auto')
                array.append(img.get_array())
            except IndexError:
                pass
        return array

    def draw_roll(self,fig,a1,filename:str,just_image=True,show=True):
        roll = self.get_roll()
        # build and set fig obj
        #plt.ioff()
        #fig = plt.figure(figsize=(7, 5),frameon=False)
        #a1 = fig.add_subplot(111)
        a1.axis("equal")
        a1.set_facecolor("black")

        # change unit of time axis from tick to second
        tick = self.get_total_ticks()
        second = mido.tick2second(tick, self.ticks_per_beat, self.get_tempo())
        if second > 10:
            x_label_period_sec = second // 10
        else:
            x_label_period_sec = second / 10  # ms
        x_label_interval = mido.second2tick(x_label_period_sec, self.ticks_per_beat, self.get_tempo()) / self.sr
        if just_image==False:                
            fig.xticks([int(x * x_label_interval) for x in range(20)], [round(x * x_label_period_sec, 2) for x in range(20)])
            # change scale and label of y axis
            #plt...
            fig.yticks([y*16 for y in range(8)], [y*16 for y in range(8)])
        else:
            fig.xticks([])
            fig.yticks([])
        # build colors
        channel_nb = 16
        transparent = ColorConverter.to_rgba('white')
        colors = [mpl.colors.to_rgba(mpl.colors.hsv_to_rgb((i / channel_nb, 1, 1)), alpha=1) for i in range(channel_nb)]
        cmaps = [mpl.colors.LinearSegmentedColormap.from_list('my_cmap', [transparent, colors[i]], 128) for i in
                 range(channel_nb)]
        # build color maps
        for i in range(channel_nb):
            cmaps[i]._init()
            # create your alpha array and fill the colormap with them.
            alphas = np.linspace(0, 1, cmaps[i].N + 3)
            # create the _lut array, with rgba values
            cmaps[i]._lut[:, -1] = alphas

        # draw piano roll and stack image on a1
        for i in range(channel_nb):
            try:
                a1.imshow(roll[i], origin="lower", interpolation='nearest', cmap=cmaps[i], aspect='auto')
                # a1.imshow(roll[i], origin="lower", interpolation='nearest', cmap='inferno', aspect='auto')
            except IndexError:
                pass
        fig.savefig(filename,dpi=200)
        a1.cla()
        fig.clf()

    def get_tempo(self):
        try:
            return self.meta["set_tempo"]["tempo"]
        except:
            return 500000

    def get_total_ticks(self):
        max_ticks = 0
        for channel in range(16):
            ticks = sum(msg.time for msg in self.events[channel])
            if ticks > max_ticks:
                max_ticks = ticks
        return max_ticks

# if __name__ == "__main__":
#     filename = '/content/drive/My Drive/Diplomado Python/Notebooks & Resources/Proyecto Final/Piano solo midi dataset/Bach/bach_847.mid'
#     mid = MidiFile(filename)
#     roll = mid.get_roll()
#     path = '/content/drive/My Drive/Diplomado Python/Notebooks & Resources/Proyecto Final/Images/'
#     filename = path+'roll_1.png'    
#     mid.draw_roll(filename)

In [9]:
filename = '/content/drive/My Drive/Diplomado Python/Notebooks & Resources/Proyecto Final/note_values.p'
with open(filename, 'rb') as fp:
    note_values = pickle.load(fp)

filename = '/content/drive/My Drive/Diplomado Python/Notebooks & Resources/Proyecto Final/cc.p'
with open(filename,'rb') as fp:
    cc = pickle.load(fp) 

dataset1_df = pd.read_pickle('/content/drive/My Drive/Diplomado Python/Notebooks & Resources/Proyecto Final/dataset1.p')
dataset2_df = pd.read_pickle('/content/drive/My Drive/Diplomado Python/Notebooks & Resources/Proyecto Final/dataset2.p')
dataset2_full_df = pd.read_pickle('/content/drive/My Drive/Diplomado Python/Notebooks & Resources/Proyecto Final/dataset2-big.p')
composers_df = pd.read_pickle('/content/drive/My Drive/Diplomado Python/Notebooks & Resources/Proyecto Final/composers.p')

In [ ]:
rootdir = '/content/drive/My Drive/Diplomado Python/Notebooks & Resources/Proyecto Final/Piano solo midi dataset'
#rootdir = '/content/drive/My Drive/Diplomado Python/Notebooks & Resources/Proyecto Final/Otro dataset de pianos midis/maestro-v2.0.0'
img_path = '/content/drive/My Drive/Diplomado Python/Notebooks & Resources/Proyecto Final/Images/'
k = 0
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        filename = os.path.join(subdir, file)
        try:
            mid_bar = MidiFile(filename)
            img_filename = img_path + file[:-3] + 'png'
            mid_bar.draw_roll(img_filename,show=False)
            # midi_info_pd.loc[idx,'Image'] = True
        except:
            print("El archivo",file,"no se puede leer")
            # midi_info_pd.loc[idx,'Broken'] = True
        # midi_info_pd.to_pickle('/content/drive/My Drive/Diplomado Python/Notebooks & Resources/Proyecto Final/dataset1.p')
        print(k)
        k += 1

In [ ]:
def msj(k,width):
    if (k+1)%width > 0:
        print(k, end='Ϟ ')
    else:
        print(k,'Ϟ',end='\r')

In [ ]:
sounds = MidiFile('/content/drive/My Drive/Diplomado Python/Notebooks & Resources/Proyecto Final/Otro dataset de pianos midis/maestro-v2.0.0/2008/MIDI-Unprocessed_02_R1_2008_01-05_ORIG_MID--AUDIO_02_R1_2008_wav--2.midi')
sounds.draw_roll('/content/drive/My Drive/Diplomado Python/Notebooks & Resources/Proyecto Final/Images/Previos/test.png')
roll = sounds.get_roll()

El roll es un arreglo de 

16 (channels) x 128 (notas) x 177712 (duración)

In [ ]:
roll.shape

(16, 128, 17712)

# Compositores y Periodos

In [ ]:
dataset2_df.to_pickle('/content/drive/My Drive/Diplomado Python/Notebooks & Resources/Proyecto Final/dataset2.p')

In [ ]:
composers_dict = {composers_df.loc[k,'last_name']:composers_df.loc[k,'period'] 
                  for k in composers_df.index.to_list()}
composers_dict

In [ ]:
arr1 = dataset1_df['Period'].values
arr2 = dataset2_df['Period'].values
arr = np.concatenate((arr1,arr2))

periodos = np.unique(arr)
data = {p:0 for p in periodos if p not in ['Modern','Nationalism'] }

for p in data.keys():
    data[p] += dataset1_df[dataset1_df['Period']==p].shape[0]
    data[p] += dataset2_df[dataset2_df['Period']==p].shape[0]
print(data)
plt.figure(dpi=150)
#plt.suptitle('Histogram')
plt.bar(list(data.keys()),list(data.values()), color='mediumblue',edgecolor='slategray')
plt.xticks(fontsize=16)
plt.yticks(fontsize=14)
plt.savefig('histogram_periods.png',dpi=150)
plt.show()

{'Baroque': 188, 'Classicism': 321, 'Romanticism': 1040}


In [ ]:
np.sum(list(data.values()))

1549

## Duraciones

In [ ]:
mid.length

296.66952770833325

In [ ]:
# path = '/content/drive/My Drive/Diplomado Python/Notebooks & Resources/Proyecto Final/Piano solo midi dataset/bach/'
path = '/content/drive/My Drive/Diplomado Python/Notebooks & Resources/Proyecto Final/Piano solo midi dataset/chopin/'
directory = os.fsencode(path)

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    midi_filename = path+filename
    midi_file = MidiFile(midi_filename, clip=True)
    print(filename,"--->",midi_file.length,"segundos --->",midi_file.length/60,"minutos.")

# Visualization

In [ ]:
mid_bar = MidiFile('/content/drive/My Drive/Diplomado Python/Notebooks & Resources/Proyecto Final/Piano solo midi dataset/bach/bach_847.mid')
mid_rom = MidiFile('/content/drive/My Drive/Diplomado Python/Notebooks & Resources/Proyecto Final/Piano solo midi dataset/chopin/chpn-p13.mid')
mid_cla = MidiFile('/content/drive/My Drive/Diplomado Python/Notebooks & Resources/Proyecto Final/Piano solo midi dataset/mozart/mz_331_2.mid')

In [ ]:
mid_bar.draw_roll('/content/drive/My Drive/Diplomado Python/Notebooks & Resources/Proyecto Final/Images/Previos/roll_bar_3.png')
# mid_cla.draw_roll('/content/drive/My Drive/Diplomado Python/Notebooks & Resources/Proyecto Final/Images/Previos/roll_cla.png')
# mid_rom.draw_roll('/content/drive/My Drive/Diplomado Python/Notebooks & Resources/Proyecto Final/Images/Previos/roll_rom.png')

In [ ]:
roll_cla = mid_cla.get_roll()
print(roll_cla.shape)

(16, 128, 107280)


Por primera vez

In [ ]:
rootdir = '/content/drive/My Drive/Diplomado Python/Notebooks & Resources/Proyecto Final/Piano solo midi dataset'
#rootdir = '/content/drive/My Drive/Diplomado Python/Notebooks & Resources/Proyecto Final/Otro dataset de pianos midis/maestro-v2.0.0'
img_path = '/content/drive/My Drive/Diplomado Python/Notebooks & Resources/Proyecto Final/Images/'
k = 0
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        filename = os.path.join(subdir, file)
        try:
            mid_bar = MidiFile(filename)
            img_filename = img_path + file[:-3] + 'png'
            mid_bar.draw_roll(img_filename,show=False)
            # midi_info_pd.loc[idx,'Image'] = True
        except:
            print("El archivo",file,"no se puede leer")
            # midi_info_pd.loc[idx,'Broken'] = True
        # midi_info_pd.to_pickle('/content/drive/My Drive/Diplomado Python/Notebooks & Resources/Proyecto Final/dataset1.p')
        print(k)
        k += 1

Continuar leyendo midis y convirtiéndolos en imagenes

In [ ]:
rootdir = '/content/drive/My Drive/Diplomado Python/Notebooks & Resources/Proyecto Final/Piano solo midi dataset'
img_path = '/content/drive/My Drive/Diplomado Python/Notebooks & Resources/Proyecto Final/Images/'
k = 0
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        filename = os.path.join(subdir, file)
        idx = midi_info_pd[midi_info_pd['root_Filename'] == file[:-4]].index.tolist()
        idx = idx[0]
        if midi_info_pd.loc[idx,'Image'] == False:
            try:
                mid_bar = MidiFile(filename)
                img_filename = img_path + file[:-3] + 'png'
                mid_bar.draw_roll(img_filename,show=False)
                midi_info_pd.loc[idx,'Image'] = True
            except:
                print("El archivo",file,"no se puede leer")
                midi_info_pd.loc[idx,'Broken'] = True
            midi_info_pd.to_pickle('/content/drive/My Drive/Diplomado Python/Notebooks & Resources/Proyecto Final/dataset1.p')
            msj(k)
        k += 1

In [ ]:
df = midi_info_pd.drop_duplicates(subset='root_Filename', keep="first")
df.shape

(314, 7)

Preparar listas para crear dataframe de nuevos archivos

In [ ]:
rootdir = '/content/drive/My Drive/Diplomado Python/Notebooks & Resources/Proyecto Final/Clementi'

midi_files = []
pathfiles = []
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        filename = os.path.join(subdir, file)
        midi_files.append(file)
        pathfiles.append(filename)
print(midi_files[-3:])
root_files = [x[:-4] for x in midi_files]
img_files = [x[:-4]+'.png' for x in midi_files]
print(root_files[-3:])
print(img_files[-3:])

['clementi_opus36_4_1.mid', 'clementi_opus36_5_2.mid', 'clementi_opus36_6_1.mid']
['clementi_opus36_4_1', 'clementi_opus36_5_2', 'clementi_opus36_6_1']
['clementi_opus36_4_1.png', 'clementi_opus36_5_2.png', 'clementi_opus36_6_1.png']


Crear dataframe con nuevos archivos

In [ ]:
colz = midi_info_pd.columns.to_list()
n = len(root_files)
m = len(colz)
comps = ['' for k in range(n)]
periods = ['' for k in range(n)]
haz_image = [False for k in range(n)]
haz_broken = [False for k in range(n)]
lists = [img_files,pathfiles,comps,periods,haz_image,haz_broken,root_files]
datos = {colz[k]:lists[k] for k in range(m)}
new_files_df = pd.DataFrame(data=datos)

new_files_df

,img_Filename,Path,Composer,Period,Image,Broken,root_Filename
0,clementi_opus36_1_2.png,/content/drive/My Drive/Diplomado Python/Noteb...,,,False,False,clementi_opus36_1_2
1,clementi_opus36_6_2.png,/content/drive/My Drive/Diplomado Python/Noteb...,,,False,False,clementi_opus36_6_2
2,clementi_opus36_5_1.png,/content/drive/My Drive/Diplomado Python/Noteb...,,,False,False,clementi_opus36_5_1
3,clementi_opus36_3_2.png,/content/drive/My Drive/Diplomado Python/Noteb...,,,False,False,clementi_opus36_3_2
4,clementi_opus36_1_1.png,/content/drive/My Drive/Diplomado Python/Noteb...,,,False,False,clementi_opus36_1_1
5,clementi_opus36_2_3.png,/content/drive/My Drive/Diplomado Python/Noteb...,,,False,False,clementi_opus36_2_3
6,clementi_opus36_2_1.png,/content/drive/My Drive/Diplomado Python/Noteb...,,,False,False,clementi_opus36_2_1
7,clementi_opus36_3_1.png,/content/drive/My Drive/Diplomado Python/Noteb...,,,False,False,clementi_opus36_3_1
8,clementi_opus36_4_2.png,/content/drive/My Drive/Diplomado Python/Noteb...,,,False,False,clementi_opus36_4_2
9,clementi_opus36_4_3.png,/content/drive/My Drive/Diplomado Python/Noteb...,,,False,False,clementi_opus36_4_3


In [ ]:
def GenerarImagenes_1stTime(rootdir,img_path,files_df,df_filename):
    k = 0
    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            filename = os.path.join(subdir, file)
            [idx] = files_df[files_df['root_Filename'] == file[:-4]].index.tolist()
            try:
                mid_bar = MidiFile(filename)
                img_filename = img_path + file[:-3] + 'png'
                mid_bar.draw_roll(img_filename,show=False)
                files_df.at[idx,'Image'] = True
            except:
                print("El archivo",file,"no se puede leer")
                files_df.at[idx,'Broken'] = True
            files_df.to_pickle(df_filename)
            print(k)
            k += 1

df_filename = '/content/drive/My Drive/Diplomado Python/Notebooks & Resources/Proyecto Final/dataset_clem.p'
rootdir = '/content/drive/My Drive/Diplomado Python/Notebooks & Resources/Proyecto Final/Clementi'
img_path = '/content/drive/My Drive/Diplomado Python/Notebooks & Resources/Proyecto Final/Images/'
GenerarImagenes_1stTime(rootdir,img_path,new_files_df,df_filename)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:138: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


In [ ]:
midi_info_pd =  midi_info_pd.append(new_files_df,ignore_index=True)
midi_info_pd
# new_files_df['Composer'] = 'Clementi'
# new_files_df

,img_Filename,Path,Composer,Period,Image,Broken,root_Filename
0,chpn_op25_e2.png,/content/drive/My Drive/Diplomado Python/Noteb...,Chopin,,True,False,chpn_op25_e2
1,chpn_op25_e4.png,/content/drive/My Drive/Diplomado Python/Noteb...,Chopin,,True,False,chpn_op25_e4
2,chp_op18.png,/content/drive/My Drive/Diplomado Python/Noteb...,Chopin,,True,False,chp_op18
3,chpn_op7_1.png,/content/drive/My Drive/Diplomado Python/Noteb...,Chopin,,True,False,chpn_op7_1
4,chpn_op35_4.png,/content/drive/My Drive/Diplomado Python/Noteb...,Chopin,,True,False,chpn_op35_4
...,...,...,...,...,...,...,...
326,clementi_opus36_1_3.png,/content/drive/My Drive/Diplomado Python/Noteb...,Clementi,,True,False,clementi_opus36_1_3
327,clementi_opus36_2_2.png,/content/drive/My Drive/Diplomado Python/Noteb...,Clementi,,True,False,clementi_opus36_2_2
328,clementi_opus36_4_1.png,/content/drive/My Drive/Diplomado Python/Noteb...,Clementi,,True,False,clementi_opus36_4_1
329,clementi_opus36_5_2.png,/content/drive/My Drive/Diplomado Python/Noteb...,Clementi,,True,False,clementi_opus36_5_2


In [ ]:
midi_info_pd.sort_values(by='Composer')

,img_Filename,Path,Composer,Period,Image,Broken,root_Filename
192,alb_se6.png,/content/drive/My Drive/Diplomado Python/Noteb...,Albéniz,,True,False,alb_se6
191,alb_se7.png,/content/drive/My Drive/Diplomado Python/Noteb...,Albéniz,,True,False,alb_se7
179,alb_esp4.png,/content/drive/My Drive/Diplomado Python/Noteb...,Albéniz,,True,False,alb_esp4
181,alb_esp1.png,/content/drive/My Drive/Diplomado Python/Noteb...,Albéniz,,True,False,alb_esp1
182,alb_esp2.png,/content/drive/My Drive/Diplomado Python/Noteb...,Albéniz,,True,False,alb_esp2
...,...,...,...,...,...,...,...
239,ty_juli.png,/content/drive/My Drive/Diplomado Python/Noteb...,Tchaikovsky,,True,False,ty_juli
240,ty_oktober.png,/content/drive/My Drive/Diplomado Python/Noteb...,Tchaikovsky,,True,False,ty_oktober
241,ty_september.png,/content/drive/My Drive/Diplomado Python/Noteb...,Tchaikovsky,,True,False,ty_september
235,ty_april.png,/content/drive/My Drive/Diplomado Python/Noteb...,Tchaikovsky,,True,False,ty_april


In [ ]:
periods_dict

{'Albéniz': 'Romanticism',
 'Bach': 'Baroque',
 'Balakirew': 'Romanticism',
 'Beethoven': 'Classicism',
 'Borodin': 'Romanticism',
 'Brahms': 'Romanticism',
 'Burgmueller': 'Romanticism',
 'Chopin': 'Romanticism',
 'Clementi': 'Classicism',
 'Debussy': 'Romanticism',
 'Godowsky': 'Romanticism',
 'Granados': 'Romanticism',
 'Grieg': 'Romanticism',
 'Haydn': 'Classicism',
 'Liszt': 'Romanticism',
 'Mendelssohn': 'Romanticism',
 'Moszkowski': 'Romanticism',
 'Mozart': 'Classicism',
 'Mussorgsky': 'Romanticism',
 'Rachmaninov': 'Romanticism',
 'Ravel': 'Romanticism',
 'Schubert': 'Romanticism',
 'Schumann': 'Romanticism',
 'Sinding': 'Romanticism',
 'Tchaikovsky': 'Romanticism'}

In [ ]:
midi_info_pd

False

In [ ]:
idxs = midi_info_pd[midi_info_pd['Composer']=='Albéniz'].index.to_list()
midi_info_pd.loc[idxs,'Composer'] = 'Albéniz'

In [ ]:
periods_dict = {composers_df.loc[k,'last_name']:composers_df.loc[k,'period'] 
                for k in range(composers_df.shape[0])}

midi_info_pd['Period'] = [periods_dict[midi_info_pd.at[k,'Composer']] for k in range(midi_info_pd.shape[0])]
midi_info_pd

,img_Filename,Path,Composer,Period,Image,Broken,root_Filename
0,chpn_op25_e2.png,/content/drive/My Drive/Diplomado Python/Noteb...,Chopin,Romanticism,True,False,chpn_op25_e2
1,chpn_op25_e4.png,/content/drive/My Drive/Diplomado Python/Noteb...,Chopin,Romanticism,True,False,chpn_op25_e4
2,chp_op18.png,/content/drive/My Drive/Diplomado Python/Noteb...,Chopin,Romanticism,True,False,chp_op18
3,chpn_op7_1.png,/content/drive/My Drive/Diplomado Python/Noteb...,Chopin,Romanticism,True,False,chpn_op7_1
4,chpn_op35_4.png,/content/drive/My Drive/Diplomado Python/Noteb...,Chopin,Romanticism,True,False,chpn_op35_4
...,...,...,...,...,...,...,...
326,clementi_opus36_1_3.png,/content/drive/My Drive/Diplomado Python/Noteb...,Clementi,Classicism,True,False,clementi_opus36_1_3
327,clementi_opus36_2_2.png,/content/drive/My Drive/Diplomado Python/Noteb...,Clementi,Classicism,True,False,clementi_opus36_2_2
328,clementi_opus36_4_1.png,/content/drive/My Drive/Diplomado Python/Noteb...,Clementi,Classicism,True,False,clementi_opus36_4_1
329,clementi_opus36_5_2.png,/content/drive/My Drive/Diplomado Python/Noteb...,Clementi,Classicism,True,False,clementi_opus36_5_2


In [ ]:
midi_info_pd.to_pickle('/content/drive/My Drive/Diplomado Python/Notebooks & Resources/Proyecto Final/dataset1.p')

# El otro dataset

In [ ]:
dataset2_df[dataset2_df['Image'] == True].shape

(378, 8)

In [ ]:
np.unique(dataset2_df['Image'].values)

array([False,  True])

In [ ]:
dataset2_df[dataset2_df['Image']==False]

,img_Filename,Path,composer,Period,Image,Broken,root_Filename,canonical_title
1,MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MID--AU...,/content/drive/My Drive/Diplomado Python/Noteb...,Berg,Romanticism,False,False,MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MID--AU...,Sonata Op. 1
2,MIDI-Unprocessed_066_PIANO066_MID--AUDIO-split...,/content/drive/My Drive/Diplomado Python/Noteb...,Berg,Romanticism,False,False,MIDI-Unprocessed_066_PIANO066_MID--AUDIO-split...,Sonata Op. 1
3,MIDI-Unprocessed_XP_21_R1_2004_01_ORIG_MID--AU...,/content/drive/My Drive/Diplomado Python/Noteb...,Scriabin,Romanticism,False,False,MIDI-Unprocessed_XP_21_R1_2004_01_ORIG_MID--AU...,"24 Preludes Op. 11, No. 13-24"
4,MIDI-Unprocessed_17_R1_2006_01-06_ORIG_MID--AU...,/content/drive/My Drive/Diplomado Python/Noteb...,Scriabin,Romanticism,False,False,MIDI-Unprocessed_17_R1_2006_01-06_ORIG_MID--AU...,"3 Etudes, Op. 65"
5,MIDI-Unprocessed_07_R1_2009_04-05_ORIG_MID--AU...,/content/drive/My Drive/Diplomado Python/Noteb...,Scriabin,Romanticism,False,False,MIDI-Unprocessed_07_R1_2009_04-05_ORIG_MID--AU...,"5 Preludes, Op.15"
...,...,...,...,...,...,...,...,...
1275,ORIG-MIDI_03_7_6_13_Group__MID--AUDIO_09_R1_20...,/content/drive/My Drive/Diplomado Python/Noteb...,Mozart,Classicism,False,False,ORIG-MIDI_03_7_6_13_Group__MID--AUDIO_09_R1_20...,"Sonata in F Major, K. 280, 1st mov."
1276,MIDI-Unprocessed_XP_14_R1_2004_04_ORIG_MID--AU...,/content/drive/My Drive/Diplomado Python/Noteb...,Mozart,Classicism,False,False,MIDI-Unprocessed_XP_14_R1_2004_04_ORIG_MID--AU...,"Sonata in F Major, K280"
1277,MIDI-Unprocessed_XP_14_R1_2004_04_ORIG_MID--AU...,/content/drive/My Drive/Diplomado Python/Noteb...,Mozart,Classicism,False,False,MIDI-Unprocessed_XP_14_R1_2004_04_ORIG_MID--AU...,"Sonata in F Major, K280"
1278,MIDI-Unprocessed_XP_14_R1_2004_04_ORIG_MID--AU...,/content/drive/My Drive/Diplomado Python/Noteb...,Mozart,Classicism,False,False,MIDI-Unprocessed_XP_14_R1_2004_04_ORIG_MID--AU...,"Sonata in F Major, K280"


In [ ]:
def GenerarImagenes_continuar(rootdir,img_path,files_df,df_filename):
    k = 0
    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            filename = os.path.join(subdir, file)
            idx = files_df[files_df['root_Filename'] == file[:-5]].index.tolist()
            k += 1
            if len(idx)==0:
                break
            else:
                idx = idx[0]
            if k-1<50 and files_df.loc[idx,'Image']==False:
                print(idx)
                print(files_df.loc[idx,'Image'])
            if files_df.loc[idx,'Image'] == False:
                try:
                    mid_bar = MidiFile(filename)
                    img_filename = img_path + files_df.loc[idx,'img_Filename'] #img_path + file[:-3] + 'png'
                    mid_bar.draw_roll(img_filename,show=False)
                    files_df.loc[idx,'Image'] = True
                except:
                    print("El archivo",file,"no se puede leer")
                    files_df.loc[idx,'Broken'] = True                
                files_df.to_pickle(df_filename)
                msj(k-1)

ruta_midis = '/content/drive/My Drive/Diplomado Python/Notebooks & Resources/Proyecto Final/Otro dataset de pianos midis/maestro-v2.0.0'
ruta_imagenes = '/content/drive/My Drive/Diplomado Python/Notebooks & Resources/Proyecto Final/Images2/'
ruta_dataframe = '/content/drive/My Drive/Diplomado Python/Notebooks & Resources/Proyecto Final/dataset2.p'

GenerarImagenes_continuar(ruta_midis,ruta_imagenes,dataset2_df,ruta_dataframe)

In [ ]:
dataset2_df.to_pickle('/content/drive/My Drive/Diplomado Python/Notebooks & Resources/Proyecto Final/dataset2.p')

Prueba de generar mil mensajes sin que se acabe la memoria

In [ ]:
arr_2d = np.sum(arr,axis=2)

# Extras

## Diccionario de notas y valores

In [ ]:
notas = ['la','sib','si','do','reb','re','mib','mi','fa','solb','sol','lab']
note_names_dict = {k:notas[(k-21)%12] for k in range(21,109)}
print(note_names_dict)

{21: 'la', 22: 'sib', 23: 'si', 24: 'do', 25: 'reb', 26: 're', 27: 'mib', 28: 'mi', 29: 'fa', 30: 'solb', 31: 'sol', 32: 'lab', 33: 'la', 34: 'sib', 35: 'si', 36: 'do', 37: 'reb', 38: 're', 39: 'mib', 40: 'mi', 41: 'fa', 42: 'solb', 43: 'sol', 44: 'lab', 45: 'la', 46: 'sib', 47: 'si', 48: 'do', 49: 'reb', 50: 're', 51: 'mib', 52: 'mi', 53: 'fa', 54: 'solb', 55: 'sol', 56: 'lab', 57: 'la', 58: 'sib', 59: 'si', 60: 'do', 61: 'reb', 62: 're', 63: 'mib', 64: 'mi', 65: 'fa', 66: 'solb', 67: 'sol', 68: 'lab', 69: 'la', 70: 'sib', 71: 'si', 72: 'do', 73: 'reb', 74: 're', 75: 'mib', 76: 'mi', 77: 'fa', 78: 'solb', 79: 'sol', 80: 'lab', 81: 'la', 82: 'sib', 83: 'si', 84: 'do', 85: 'reb', 86: 're', 87: 'mib', 88: 'mi', 89: 'fa', 90: 'solb', 91: 'sol', 92: 'lab', 93: 'la', 94: 'sib', 95: 'si', 96: 'do', 97: 'reb', 98: 're', 99: 'mib', 100: 'mi', 101: 'fa', 102: 'solb', 103: 'sol', 104: 'lab', 105: 'la', 106: 'sib', 107: 'si', 108: 'do'}


In [ ]:
for num,note in zip(note_names_dict.keys(),note_names_dict.values()):
    k = num - 21
    if k>2:
        alt = 1 + ((k-3)//12)
        note_names_dict[num] += str(alt)
note_names_dict[21],note_names_dict[22],note_names_dict[23] = 'la0','sib0','si0'
print(note_names_dict) 

{21: 'la0', 22: 'sib0', 23: 'si0', 24: 'do1', 25: 'reb1', 26: 're1', 27: 'mib1', 28: 'mi1', 29: 'fa1', 30: 'solb1', 31: 'sol1', 32: 'lab1', 33: 'la1', 34: 'sib1', 35: 'si1', 36: 'do2', 37: 'reb2', 38: 're2', 39: 'mib2', 40: 'mi2', 41: 'fa2', 42: 'solb2', 43: 'sol2', 44: 'lab2', 45: 'la2', 46: 'sib2', 47: 'si2', 48: 'do3', 49: 'reb3', 50: 're3', 51: 'mib3', 52: 'mi3', 53: 'fa3', 54: 'solb3', 55: 'sol3', 56: 'lab3', 57: 'la3', 58: 'sib3', 59: 'si3', 60: 'do4', 61: 'reb4', 62: 're4', 63: 'mib4', 64: 'mi4', 65: 'fa4', 66: 'solb4', 67: 'sol4', 68: 'lab4', 69: 'la4', 70: 'sib4', 71: 'si4', 72: 'do5', 73: 'reb5', 74: 're5', 75: 'mib5', 76: 'mi5', 77: 'fa5', 78: 'solb5', 79: 'sol5', 80: 'lab5', 81: 'la5', 82: 'sib5', 83: 'si5', 84: 'do6', 85: 'reb6', 86: 're6', 87: 'mib6', 88: 'mi6', 89: 'fa6', 90: 'solb6', 91: 'sol6', 92: 'lab6', 93: 'la6', 94: 'sib6', 95: 'si6', 96: 'do7', 97: 'reb7', 98: 're7', 99: 'mib7', 100: 'mi7', 101: 'fa7', 102: 'solb7', 103: 'sol7', 104: 'lab7', 105: 'la7', 106: 'sib

In [ ]:
filename = '/content/drive/My Drive/Diplomado Python/Notebooks & Resources/Proyecto Final/note_values.p'
with open(filename, 'wb') as fp:
    pickle.dump(note_names_dict, fp)

## CC messages dictionary

In [ ]:
cc_df = pd.read_excel('/content/drive/My Drive/Diplomado Python/Notebooks & Resources/Proyecto Final/CC.xlsx')
cc_df.head()

,CC,Meaning,Details
0,0,Bank Select,Allows user to switch bank for patch selection...
1,1,Modulation,Generally this CC controls a vibrato effect (p...
2,2,Breath Controller,Often times associated with aftertouch message...
3,3,Undefined,
4,4,Foot Controller,Often used with aftertouch messages. It can se...


In [ ]:
cc = {cc_df.loc[k,'CC']:cc_df.loc[k,'Meaning'] for k in range(cc_df.shape[0])} 

In [ ]:
cc

{0: 'Bank Select ',
 1: 'Modulation ',
 2: 'Breath Controller ',
 3: 'Undefined ',
 4: 'Foot Controller ',
 5: 'Portamento',
 6: 'Data Entry Most Significant Bit(MSB) ',
 7: 'Volume ',
 8: 'Balance ',
 9: 'Undefined ',
 10: 'Pan ',
 11: 'Expression ',
 12: 'Effect Controller 1 ',
 13: 'Effect Controller 2 ',
 14: 'Undefined ',
 15: 'Undefined ',
 64: 'Damper Pedal /',
 65: 'Portamento On/Off Switch ',
 66: 'Sostenuto On/Off Switch ',
 67: 'Soft Pedal On/Off Switch ',
 68: 'Legato FootSwitch ',
 69: 'Hold 2 ',
 70: 'Sound Controller 1 ',
 71: 'Sound Controller 2 ',
 72: 'Sound Controller 3 ',
 73: 'Sound Controller 4 ',
 74: 'Sound Controller 5 ',
 75: 'Sound Controller 6 ',
 76: 'Sound Controller 7 ',
 77: 'Sound Controller 8 ',
 78: 'Sound Controller 9 ',
 79: 'Sound Controller 10 ',
 80: 'General Purpose',
 81: 'General Purpose MIDI CC Controller ',
 82: 'General Purpose',
 83: 'General Purpose MIDI CC Controller ',
 84: 'Portamento CC Control ',
 91: 'Effect 1 Depth ',
 92: 'Effect 

In [ ]:
filename = '/content/drive/My Drive/Diplomado Python/Notebooks & Resources/Proyecto Final/cc.p'
with open(filename, 'wb') as fp:
    pickle.dump(note_names_dict, fp)

## Manipular los archivos

In [ ]:
filename = '/content/drive/My Drive/Diplomado Python/Notebooks & Resources/Proyecto Final/Piano solo midi dataset/tschai/ty_august.mid'
mid = mido.MidiFile(filename, clip=True)
print(mid)
for track in mid.tracks:
    print(track)

<midi file '/content/drive/My Drive/Diplomado Python/Notebooks & Resources/Proyecto Final/Piano solo midi dataset/tschai/ty_august.mid' type 1, 8 tracks, 6271 messages>
<midi track 'Die Jahreszeiten' 801 messages>
<midi track 'Piano right' 2861 messages>
<midi track 'Piano left' 2600 messages>
<midi track 'Tschaikowsky: The Seasons: August' 2 messages>
<midi track 'Copyright © 2004 by Bernd Krüger' 2 messages>
<midi track 'http://www.piano-midi.de' 2 messages>
<midi track 'Edition: 2008-09-21' 2 messages>
<midi track '' 1 messages>


In [ ]:
for k,msg in enumerate(mid.tracks[1]):
    #print(msg.type)
    non_allowed = ['text','midi_port','track_name','program_change','control_change']
    if msg.type not in non_allowed:
        if k>2 and k<30 and msg.velocity>0:
            try:
                print(note_values[msg.note],"vel:",msg.velocity,"time:",msg.time)
            except:
                print(msg.type,"-",cc[msg.control],"- time:",msg.time)

sol4 vel: 56 time: 241
do5 vel: 60 time: 0
mi5 vel: 63 time: 0
sol4 vel: 50 time: 12
do5 vel: 47 time: 0
mi5 vel: 47 time: 0
sol4 vel: 56 time: 240
do5 vel: 60 time: 0
mi5 vel: 62 time: 0
sol4 vel: 50 time: 12
do5 vel: 48 time: 0
mi5 vel: 48 time: 0


In [ ]:
times = []
notes = []
for k,msg in enumerate(mid.tracks[1]):
    if msg.type == 'note_on':
        times.append(msg.time)
        notes.append(msg.note)
        #print(k,msg.type,msg.note,msg.time)
print("Right hand:")
print(np.unique(times),"\n ----")
notas = np.unique(notes)
notas_nombres = [note_values[x] for x in notas]
print("Notes\n",notas_nombres,"\n",np.unique(notes).shape[0])

times = []
notes = []
for k,msg in enumerate(mid.tracks[2]):
    if msg.type == 'note_on':
        times.append(msg.time)
        notes.append(msg.note)
        #print(k,msg.type,msg.note,msg.time)
print("Left hand:")
print(np.unique(times),"\n ----")
notas = np.unique(notes)
notas_nombres = [note_values[x] for x in notas]
print("Notes\n",notas_nombres,"\n",np.unique(notes).shape[0])

In [ ]:
for k,msg in enumerate(mid.tracks[1]):
    if msg.type == 'note_on':
        print(k,note_values[msg.note],msg.time)